In [1]:
# ================================ #
#      호선별 거점역/시간/요일      #
# ================================ #

target_stations_group = {
    "1호선": ['서울역','시청역','종각역','종로3가역','종로5가역','동대문역','신설동역','제기동역','청량리역','동묘앞역'],
    "2호선": ['시청역','을지로입구역','을지로3가역','을지로4가역','동대문역사문화공원역','신당역','상왕십리역','왕십리역','한양대역','뚝섬역','성수역','건대입구역','구의역','강변역','잠실나루역','잠실역','잠실새내역','종합운동장역','삼성역','선릉역','역삼역','강남역','교대역','서초역','방배역','사당역','낙성대역','서울대입구역','봉천역','신림역','신대방역','구로디지털단지역','대림역','신도림역','문래역','영등포구청역','당산역','합정역','홍대입구역','신촌(지하)역','이대역','아현역','충정로역','용답역','신답역','신설동역','도림천역','양천구청역','신정네거리역','용두역'],
    "3호선": ['지축','구파발','연신내','불광','녹번','홍제','무악재','독립문','경복궁','안국','종로3가','을지로3가','충무로','동대입구','약수','금호','옥수','압구정','신사','잠원','고속터미널','교대','남부터미널','양재','매봉','도곡','대치','학여울','대청','일원','수서','가락시장','경찰병원','오금'],
    "4호선": ['상계','노원','창동','쌍문','수유','미아','미아삼거리','길음','성신여대입구','한성대입구','혜화','동대문','동대문역사문화공원','충무로','명동','회현','서울역','숙대입구','삼각지','신용산','이촌','동작','총신대입구','사당','남태령'],
    "5호선": ['개화산','김포공항','송정','마곡','발산','우장산','화곡','까치산','신정','목동','오목교','양평','영등포구청','영등포시장','신길','여의도','여의나루','마포','공덕','애오개','충정로','서대문','광화문','종로3가','을지로4가','동대문역사문화공원','청구','신금호','행당','왕십리','마장','답십리','장한평','군자','아차산','광나루','천호','강동','길동','굽은다리','명일','고덕','상일동','둔촌동','올림픽공원(한국체대)','방이','오금','개롱','거여','마천','강일','미사','하남풍산','하남시청','하남검단산'],
    "6호선": ['응암','새절','증산','디지털미디어시티','월드컵경기장','마포구청','망원','합정','상수','광흥창','대흥','공덕','효창공원앞','삼각지','녹사평','이태원','한강진','버티고개','약수','청구','신당','동묘앞','창신','보문','안암','고려대','월곡','상월곡','돌곶이','석계','태릉입구','화랑대','봉화산','신내'],
    "7호선": ['도봉산','수락산','마들','노원','중계','하계','공릉','태릉입구','먹골','중화','상봉','면목','사가정','용마산','중곡','군자','어린이대공원','건대입구','자양(뚝섬한강공원)','청담','강남구청','학동','논현','반포','고속터미널','내방','총신대입구','남성','숭실대입구','상도','장승배기','신대방삼거리','보라매','신풍','대림','남구로','가산디지털단지','철산','광명사거리','천왕','온수'],
    "8호선": ['천호','강동구청','몽촌토성','잠실','석촌','송파','가락시장','문정','장지','복정','산성','남한산성입구','단대오거리','신흥','수진','모란']
}

# 06 ~ 23시(=18개): ["06","07",...,"23"]
HOUR_LIST = [f"{h:02d}" for h in range(6, 24)]

# 월요일(MON), 토요일(SAT) 영어-한글 페어
DOW_LIST = [("MON", "월요일"), ("SAT", "토요일")]

# 1회 실행 최대 호출 한도
API_LIMIT = 3000

# 발급받은 appkey 입력!
API_KEY = "GmSL8Sudl92FLRkbZaTok3GCqKvHyzk57NotL9a9"

In [2]:
import requests
import pandas as pd
import os
import time

# 기본 설정
HEADERS = {"appkey": API_KEY}
STATION_META_URL = "https://apis.openapi.sk.com/puzzle/subway/meta/stations"
CONGESTION_API_URL_FMT = (
    "https://apis.openapi.sk.com/puzzle/subway/congestion/stat/car/stations/{}"
)
OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def fetch_all_stations():
    """모든 역의 메타정보를 받아 (호선, 역명) → 역코드 매핑."""
    stations = []
    offset = 0
    limit = 100
    total_count = None
    while True:
        params = {"offset": offset, "limit": limit}
        r = requests.get(STATION_META_URL, headers=HEADERS, params=params)
        r.raise_for_status()
        data = r.json()
        if total_count is None:
            total_count = int(data["status"]["totalCount"])
        stations.extend(data.get("contents", []))
        offset += limit
        if len(stations) >= total_count:
            break
        time.sleep(0.03)
    return stations


# 역 meta 데이터 로딩 및 (호선, 역명) 기준 코드 매핑
print("역 메타데이터 로딩중...")
all_stations = fetch_all_stations()
station_code_map = {}
for st in all_stations:
    key = (st["subwayLine"], st["stationName"])
    station_code_map[key] = st["stationCode"]
print(f"전체 역 수 : {len(all_stations)}")

역 메타데이터 로딩중...
전체 역 수 : 555


In [3]:
def fetch_congestion_10min_block(station_code, dow, hh):
    """
    역코드, 요일, 시간별로 API 호출→10분 단위 여러 구간 데이터 리스트 반환.
    한 번 호출로 stat.data 리스트(= 1시간 내 6~7개 10분 구간) 전개.
    """
    url = CONGESTION_API_URL_FMT.format(station_code)
    params = {"dow": dow, "hh": hh}
    r = requests.get(url, headers=HEADERS, params=params)
    r.raise_for_status()
    contents = r.json().get("contents", None)
    rows = []
    if not contents or "stat" not in contents:
        return rows
    for stat_block in contents["stat"]:
        for d in stat_block["data"]:
            rows.append(
                {
                    "호선": contents["subwayLine"],
                    "역명": contents["stationName"],
                    "역코드": contents["stationCode"],
                    "요일": dow,
                    "시간": d["hh"],
                    "분": d["mm"],
                    "상하행": stat_block.get("updnLine", None),
                    "직통여부": stat_block.get("directAt", None),
                    "구간시작역코드": stat_block.get("startStationCode", ""),
                    "구간시작역": stat_block.get("startStationName", ""),
                    "구간종료역코드": stat_block.get("endStationCode", ""),
                    "구간종료역": stat_block.get("endStationName", ""),
                    "직전역코드": stat_block.get("prevStationCode", ""),
                    "직전역": stat_block.get("prevStationName", ""),
                    "congestionCar": d["congestionCar"],
                }
            )
    return rows

In [4]:
def collect_subway_congestion_for_line(
    line, station_names, dow_list, hour_list, api_limit=3000
):
    """
    * 주어진 호선에 대해 각 거점역, 모든 요일, 모든 시간대의 혼잡도 데이터를 API 한도 내 수집
    * 수집된 데이터와 실제/예상 호출 수 반환
    """
    # (호선, 역명) 쌍 중 실제 코드가 매핑되는 역만 필터
    available_stations = [
        (name, station_code_map.get((line, name)))
        for name in station_names
        if station_code_map.get((line, name))
    ]
    tgt_station_count = len(available_stations)
    예상호출수 = tgt_station_count * len(dow_list) * len(hour_list)
    print(f"\n=== {line} ===")
    print(
        f"대상역: {tgt_station_count}개, 요일: {len(dow_list)}개, 시간: {len(hour_list)}개"
    )
    print(f"예상 API 호출 수: {예상호출수} (한도 {api_limit})")

    data_rows = []
    call_cnt = 0
    stop_flag = False

    # 반복 (요일, 시간, 역)
    for dow_en, dow_kr in dow_list:
        for hh in hour_list:
            for station_name, code in available_stations:
                if call_cnt >= api_limit:
                    print(f"API 호출수 {call_cnt}회 도달, 중단.")
                    stop_flag = True
                    break
                try:
                    rows = fetch_congestion_10min_block(code, dow_en, hh)
                    call_cnt += 1
                    for row in rows:
                        row["요일(한글)"] = dow_kr
                        data_rows.append(row)
                except Exception as e:
                    print(f"실패: {line}-{station_name} {dow_en} {hh}: {e}")
            if stop_flag:
                break
        if stop_flag:
            break
    print(f"실제 호출된 API 횟수: {call_cnt}")
    return pd.DataFrame(data_rows), call_cnt, 예상호출수

In [5]:
# 각 호선별 예상 호출수 미리 확인
print("호선별 예상 호출 수:")
for line, station_names in target_stations_group.items():
    count = len(station_names) * len(DOW_LIST) * len(HOUR_LIST)
    print(
        f"{line}: {len(station_names)}역 × {len(DOW_LIST)}요일 × {len(HOUR_LIST)}시간 = {count}회"
    )

호선별 예상 호출 수:
1호선: 10역 × 2요일 × 18시간 = 360회
2호선: 50역 × 2요일 × 18시간 = 1800회
3호선: 34역 × 2요일 × 18시간 = 1224회
4호선: 25역 × 2요일 × 18시간 = 900회
5호선: 55역 × 2요일 × 18시간 = 1980회
6호선: 34역 × 2요일 × 18시간 = 1224회
7호선: 41역 × 2요일 × 18시간 = 1476회
8호선: 16역 × 2요일 × 18시간 = 576회


In [6]:
# 2호선 메타 역명 전체 리스트 출력
for st in all_stations:
    if st["subwayLine"] == "1호선":
        print(st["stationName"])

소요산역
청산역
전곡역
연천역
동두천역
보산역
동두천중앙역
지행역
덕정역
덕계역
양주역
녹양역
가능역
의정부역
회룡역
망월사역
도봉산역
도봉역
방학역
창동역
녹천역
월계역
광운대역
석계역
신이문역
외대앞역
회기역
청량리역
제기동역
신설동역
동묘앞역
동대문역
종로5가역
종로3가역
종각역
시청역
서울역
남영역
용산역
노량진역
대방역
신길역
영등포역
신도림역
구로역
구일역
개봉역
오류동역
온수역
역곡역
소사역
부천역
중동역
송내역
부개역
부평역
백운역
동암역
간석역
주안역
도화역
제물포역
도원역
동인천역
인천역
가산디지털단지역
독산역
금천구청역
광명역
석수역
관악역
안양역
명학역
금정역
군포역
당정역
의왕역
성균관대역
화서역
수원역
세류역
병점역
서동탄역
세마역
오산대역
오산역
진위역
송탄역
서정리역
평택지제역
평택역
성환역
직산역
두정역
천안역
봉명역
쌍용역
아산역
탕정역
배방역
온양온천역
신창역


In [7]:
# ========== 아래 변수(selected_line)만 바꿔서 반복 실행 ==========

selected_line = "1호선"  # <- 1호선, 2호선 ... 8호선 중 하나 입력

df, call_cnt, 예상호출수 = collect_subway_congestion_for_line(
    selected_line, target_stations_group[selected_line], DOW_LIST, HOUR_LIST, API_LIMIT
)

if not df.empty:
    for _, dow_kr in DOW_LIST:
        # 요일별로 나눠 저장
        df_sub = df[df["요일(한글)"] == dow_kr]
        if not df_sub.empty:
            fname = f"{OUTPUT_DIR}/{selected_line}_{dow_kr}_혼잡도_거점역.csv"
            df_sub.to_csv(fname, index=False, encoding="utf-8-sig")
            print(f"[저장] {fname} ({len(df_sub)} rows)")
else:
    print("수집된 데이터가 없습니다.")


=== 1호선 ===
대상역: 10개, 요일: 2개, 시간: 18개
예상 API 호출 수: 360 (한도 3000)
실제 호출된 API 횟수: 360
[저장] output/1호선_월요일_혼잡도_거점역.csv (64152 rows)
[저장] output/1호선_토요일_혼잡도_거점역.csv (55836 rows)
